<a href="https://colab.research.google.com/github/Rahasyapandey/-Intrusion-Detection-System-IDS-with-Machine-Learning/blob/main/Intrusion_Detection_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split #for splitting data
from sklearn.preprocessing import StandardScaler #for feature scaling
from sklearn.ensemble import RandomForestClassifier #first IDS model
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [3]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [5]:
import os
print(os.listdir())



['.config', 'archive.zip', 'sample_data']


In [6]:
import zipfile

zip_path = "archive.zip"   # this is the file you see in os.listdir()
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("nsl_kdd")   # create folder nsl_kdd with all contents


In [7]:
import os
print(os.listdir("nsl_kdd"))


['KDDTest+.arff', 'KDDTrain+_20Percent.arff', 'KDDTrain+.arff', 'KDDTest-21.arff', 'KDDTrain1.jpg', 'KDDTest-21.txt', 'nsl-kdd', 'KDDTrain+_20Percent.txt', 'index.html', 'KDDTrain+.txt', 'KDDTest+.txt', 'KDDTest1.jpg']


In [8]:
import os
print(os.listdir("nsl_kdd/nsl-kdd"))


['KDDTest+.arff', 'KDDTrain+_20Percent.arff', 'KDDTrain+.arff', 'KDDTest-21.arff', 'KDDTrain1.jpg', 'KDDTest-21.txt', 'KDDTrain+_20Percent.txt', 'index.html', 'KDDTrain+.txt', 'KDDTest+.txt', 'KDDTest1.jpg']


In [9]:
train_path = "nsl_kdd/nsl-kdd/KDDTrain+.txt"

df = pd.read_csv(
    train_path,
    header=None
)
print(df.shape)
df.head()


(125973, 43)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [12]:
print(df.shape)      # should show (125973, 42)
print(len(df.columns))


(125973, 43)
43


In [14]:
train_path = "nsl_kdd/KDDTrain+.txt"   # or "nsl_kdd/nsl-kdd/KDDTrain+.txt" if that is your path
df = pd.read_csv(train_path, header=None)

print("shape:", df.shape)
print("number of columns:", len(df.columns))
df.head()


shape: (125973, 43)
number of columns: 43


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [17]:
print("df shape:", df.shape)
print("df columns:", len(df.columns))
print("columns list length:", len(columns))


df shape: (125973, 43)
df columns: 43
columns list length: 42


In [18]:
# Add a temporary name for the missing column
columns.append("extra_col")   # any name is fine, just to have 43 names

print("columns list length AFTER append:", len(columns))  # should now be 43

# Now assign
df.columns = columns
df.head()


columns list length AFTER append: 43


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,extra_col
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [21]:
# 1) Features and label from df
X = df.drop("label", axis=1)

# Binary target: 1 = attack, 0 = normal
y = (df["label"] != "normal").astype(int)

# 2) One‑hot encode categorical features
categorical_cols = ["protocol_type", "service", "flag"]
X_encoded = pd.get_dummies(X, columns=categorical_cols)

print("X_encoded shape:", X_encoded.shape)
print("y shape:", y.shape)


X_encoded shape: (125973, 123)
y shape: (125973,)


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Model
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)

# Evaluation
y_pred = rf.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9995634054375868
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13469
           1       1.00      1.00      1.00     11726

    accuracy                           1.00     25195
   macro avg       1.00      1.00      1.00     25195
weighted avg       1.00      1.00      1.00     25195



In [ ]:
# Intrusion Detection System using Machine Learning (NSL-KDD)

#This notebook builds a machine-learning based IDS using the NSL-KDD dataset:
#- Loads and preprocesses NSL-KDD data
#- Trains a Random Forest classifier
#- Evaluates performance with accuracy, classification report, and confusion matrix


In [23]:
import joblib

# Save model and scaler
joblib.dump(rf, "ids_random_forest.joblib")
joblib.dump(scaler, "ids_scaler.joblib")
joblib.dump(X_encoded.columns.tolist(), "ids_feature_names.joblib")


['ids_feature_names.joblib']

In [24]:
from google.colab import files

files.download("ids_random_forest.joblib")
files.download("ids_scaler.joblib")
files.download("ids_feature_names.joblib")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
# Take one sample from test set and predict
sample = X_test.iloc[[0]]
sample_scaled = scaler.transform(sample)
pred = rf.predict(sample_scaled)[0]

print("True label (0=normal,1=attack):", y_test.iloc[0])
print("Predicted label:", pred)


True label (0=normal,1=attack): 1
Predicted label: 1


In [26]:
# Taking one sample from the test set
sample = X_test.iloc[[0]]
true_label = y_test.iloc[0]

# Scale and predict
sample_scaled = scaler.transform(sample)
pred = rf.predict(sample_scaled)[0]

print("True label (0=normal, 1=attack):", true_label)
print("Predicted label:", pred)


True label (0=normal, 1=attack): 1
Predicted label: 1


In [27]:
import joblib

# Save to files in Colab
joblib.dump(rf, "ids_random_forest.joblib")
joblib.dump(scaler, "ids_scaler.joblib")
joblib.dump(X_encoded.columns.tolist(), "ids_feature_names.joblib")


['ids_feature_names.joblib']

In [28]:
from google.colab import files

# Download them to your laptop
files.download("ids_random_forest.joblib")
files.download("ids_scaler.joblib")
files.download("ids_feature_names.joblib")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>